In [1]:
# Core python libraries
import csv
import re
import math
import os
import time

# External libraries.
import nltk
import numpy
import pandas
import scipy
import sklearn
import tempfile

# Tensorflow and related.
import tensorflow as tf

# Plotting
import matplotlib.pyplot as plt

# So you know when this code block finishes.
print ("Done")

Done


In [2]:
#TODO(Max): make a library that does all this preprocessing
data = pandas.read_csv("data/train.csv") # Can read a subset. First nrows of the total.
LABEL_COLUMN = "SalaryNormalized"

TEST_SIZE = .3

print(len(data))

171482


In [3]:
# sklearn.cross_validation has been replaced with model_selection.
X_train_index, X_test_index, Y_train, Y_test = sklearn.model_selection.train_test_split(
    data.index, data[LABEL_COLUMN], test_size=TEST_SIZE, random_state=42)

# Keep train and test as pandas dataframes.
X_train = data.iloc[X_train_index]
X_test = data.iloc[X_test_index]

yb_train = numpy.array(X_train[LABEL_COLUMN].astype(numpy.float32))
yb_test= numpy.array(X_test[LABEL_COLUMN].astype(numpy.float32))

print ("Done")

Done


In [4]:
# Normalizing input values to mean 0, standard deviation 1.
train_mean, train_std = yb_train.mean(), yb_train.std()

def normalize_input(input_vector, train_mean, train_std):
    return (input_vector - train_mean)/train_std

def unnormalize_input(normalized_input_vector, train_mean, train_std):
    return (normalized_input_vector * train_std) + train_mean

# Move into a unit test.
test = numpy.arange(10)
test_mean, test_std = test.mean(), test.std()
print(test)
normed = normalize_input(test, test_mean, test_std)
print(normed)
unnormed = unnormalize_input(normed, test_mean, test_std)
print(unnormed)


# Normalize input labels and expectations
yb_train = normalize_input(yb_train, train_mean, train_std)
yb_test = normalize_input(yb_test, train_mean, train_std)


[0 1 2 3 4 5 6 7 8 9]
[-1.5666989  -1.21854359 -0.87038828 -0.52223297 -0.17407766  0.17407766
  0.52223297  0.87038828  1.21854359  1.5666989 ]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]


In [5]:
print(train_std)
print(yb_test)

17698.3
[ 0.89645004 -0.66550416  3.22435188 ..., -0.29010177  0.89645004
  0.95295256]


In [6]:
target_column = "CompleteJobListingStemmed"

x_train_text = X_train[target_column]
x_test_text = X_test[target_column]

max_document_length = max([len(x.split(" ")) for x in x_train_text])

In [7]:
# Look here for a tutorial https://medium.com/@ilblackdragon/tensorflow-text-classification-615198df9231
min_word_frequency = 75

vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
    max_document_length, min_frequency=min_word_frequency)
x_train = numpy.array(list(vocab_processor.fit_transform(x_train_text)))
print("x_train shape: %s, %s" % x_train.shape)
x_test = numpy.array(list(vocab_processor.transform(x_test_text)))
print("x_test shape: %s, %s" % x_test.shape)
n_words = len(vocab_processor.vocabulary_)
print("Total distinct words: %d" % n_words)

# Try looking at https://github.com/nfmcclure/tensorflow_cookbook/blob/master/07_Natural_Language_Processing/02_Working_with_Bag_of_Words/02_bag_of_words.py


x_train shape: 120037, 1368
x_test shape: 51445, 1368
Total distinct words: 8712


In [8]:
print(len(x_train))

120037


In [9]:
# Function to generate a training batch.

# Are these breaking past a certain point?
def generate_batch(batch_size, number_of_batches, input_data, input_labels):
    """Give me a docstring to explain."""
    batch = numpy.ndarray(shape=(batch_size, max_document_length),
                          dtype=numpy.int32)
    labels = numpy.ndarray(shape=(batch_size, 1), dtype=numpy.float32)
    
    batch_index = 0
    while batch_index < number_of_batches:
        for element_index in range(batch_size):
            # Prevents overflow.
            data_index = ((batch_index * batch_size) + element_index) % len(input_data)
            batch[element_index] = numpy.array(input_data[data_index])
            labels[element_index] = input_labels[data_index]
        batch_index += 1
        yield batch, labels
    

def input_fn():
    for batch_input, batch_labels in generate_batch(batch_size, number_of_batches,
                                                    x_train, yb_train):
        yield {x_data: batch_input, y_target: batch_labels}

def eval_input_fn():
    for batch_input, batch_labels in generate_batch(batch_size, number_of_validation_batches,
                                                    x_test, yb_test):
        yield {x_data: batch_input, y_target: batch_labels}


In [16]:
graph = tf.Graph()

sess = tf.Session(graph=graph)

learning_rate = .01

l1_regularization_coef = 0.0
l2_regularization_coef = 0.001

momentum = None

# Sklearn shuffles after epoch by default.

# if not RMSE then MAE.
HUBER = False
RMSE = True

#Implement this by batch.
batch_size = 256

#Size of input dictionary. Number of distinct words.
vocab_size = len(vocab_processor.vocabulary_)

def huber_loss(labels, predictions, delta=1.0):
 residual = tf.abs(predictions - labels)
 condition = tf.less(residual, delta)
 small_res = 0.5 * tf.square(residual)
 large_res = delta * residual - 0.5 * tf.square(delta)
 return tf.reduce_mean(tf.where(condition, small_res, large_res))

with graph.as_default():

    with tf.name_scope("data"):
        # Initialize data placeholders
        x_data = tf.placeholder(shape=[batch_size, max_document_length], dtype=tf.int32)
        y_target = tf.placeholder(shape=[batch_size, 1], dtype=tf.float32, name="labels")

    with tf.name_scope("bag_of_words"):
        with tf.device('/cpu:0'):
            identity_mat = tf.diag(tf.ones(shape=[vocab_size]))
            # This is also an embedding_lookup_sparse function that would create a sparse tensor.
            x_embed = tf.nn.embedding_lookup(identity_mat, x_data)
            # Collapse across the document length dimension. Leaving only the number of tokens.
            bag_of_words = tf.convert_to_tensor_or_sparse_tensor(tf.reduce_sum(x_embed, 1))

            print("Woohoo this looks like my bag of words shape :D %s" % bag_of_words.shape)

    # Create variables for regression
    A = tf.Variable(tf.zeros(shape=[vocab_size, 1]))
    b = tf.Variable(tf.zeros(shape=[1]))

    # A = tf.Variable(tf.random_normal(shape=[vocab_size, 1], stddev=.05))
    # Intercept term. A scalar.
    # b = tf.Variable(tf.random_normal(shape=[1], stddev=.05))

    # Model output
    product = tf.matmul(bag_of_words, A)
    model_output = tf.add(product, b, name="predictions")

    with tf.name_scope("loss"):

        # TODO(Max) try root mean square error. Should this be merely mean square error?
        if HUBER:
            loss = huber_loss(y_target, model_output)
        elif RMSE:
            loss = tf.sqrt(tf.reduce_mean(tf.pow(tf.subtract(y_target, model_output), 2)))
        else:
            # Mean Absolute Error.
            loss = tf.reduce_mean(tf.abs(tf.subtract(y_target, model_output)))

        # If we are using regularization.
        if l1_regularization_coef > 0 or l2_regularization_coef > 0:
            l1_regularizer = tf.contrib.layers.l1_regularizer(scale=l1_regularization_coef)
            l2_regularizer = tf.contrib.layers.l2_regularizer(scale=l2_regularization_coef)

            weights = tf.trainable_variables() # all vars of your graph
            regularization_penalty = tf.add(tf.contrib.layers.apply_regularization(l1_regularizer, weights),
                                             tf.contrib.layers.apply_regularization(l2_regularizer, weights))
            loss = loss + regularization_penalty # this loss needs to be minimized

    with tf.name_scope("reporting"):
        # Converts back to original, salary scale.
        error_salary_scale = tf.multiply(tf.subtract(y_target, model_output), train_std)

        mean_absolute_error_salary_scale = tf.reduce_mean(
            tf.abs(error_salary_scale))

        # Log for tensorboard
        training_summary = tf.summary.scalar('train_loss', loss)
        validation_summary = tf.summary.scalar('validation_loss', loss)
        # Add mean absolute errors.
        training_mae = tf.summary.scalar('train_mae', mean_absolute_error_salary_scale)
        validation_mae = tf.summary.scalar('validation_mae', mean_absolute_error_salary_scale)

    global_step = tf.Variable(0, name='global_step', trainable=False)

    # Declare optimizer
    if momentum is None:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    if momentum is not None:
       optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    
    # This one seems to work better :D
    train_op = tf.contrib.layers.optimize_loss(      
     loss, tf.contrib.framework.get_global_step(),      
     optimizer='Adam'
        , learning_rate=learning_rate, clip_gradients=1.0)


Woohoo this looks like my bag of words shape :D (256, 8712)
INFO:tensorflow:Scale of 0 disables regularizer.


In [31]:
# Number of
number_of_epochs = 20
batches_per_epoch = len(x_train_text)/batch_size
number_of_batches = math.ceil(len(x_train_text)/batch_size)
number_of_validation_batches = number_of_batches

logs_dir = "logs/bagofwords"

checkpoint_frequency = 50

reporting_frequency = 3

total_loss = 0
validation_loss = 0
validation_batch_average_window = 100

ckpt = None
ckpt = tf.train.get_checkpoint_state(logs_dir)  

In [32]:
# Where to save this model.
model_path = os.path.join(logs_dir, time.strftime("%Y-%m-%d-%H-%M-%S"))
                    

if ckpt:
    restore_from_model = True
    print("Restore from model!")
else:
    restore_from_model = False

with tf.Session(graph=graph, config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True)) as session:
    summaries = tf.summary.merge_all()                                                       
    writer = tf.summary.FileWriter(model_path,
        graph=session.graph)                  

    saver = tf.train.Saver(tf.global_variables()) 
                                                                                                  
    session.run(tf.global_variables_initializer())
    
    
    if restore_from_model:
        saver.restore(session, ckpt.model_checkpoint_path)
        
    global_step = 0
        
    for epoch in range(number_of_epochs):
        eval_feed_generator = eval_input_fn()
        if epoch > 0:
            # TODO(max): Shuffle
            pass

        for step, feed_dict in enumerate(input_fn()):
            global_step += 1
            start_time = time.time()

            _, temp_loss = session.run([train_op, loss], feed_dict=feed_dict)
            
            duration = time.time() - start_time
            
            print("Epoch: %s Iteration: %s : Loss: %.4f (%.3f sec)" % 
                  (epoch, (step + 1), temp_loss, duration))

            train_summ, train_mae = session.run([training_summary, training_mae], feed_dict=feed_dict)
            writer.add_summary(train_summ, global_step)
            writer.add_summary(train_mae, global_step)
            _, valid_summ, valid_mae = session.run(
                    [loss, validation_summary, validation_mae],
                    feed_dict=next(eval_feed_generator))
            writer.add_summary(valid_summ, global_step)
            writer.add_summary(valid_mae, global_step)
                
            writer.flush()                

            if (global_step % checkpoint_frequency == 0) or (global_step == number_of_batches):
                print("Saving at epoch %s step: %s" % (epoch, step + 1))
                saver.save(session, model_path, global_step=tf.contrib.framework.get_global_step())

    



Restore from model!
INFO:tensorflow:Restoring parameters from logs/bagofwords/2017-06-15-22-55-35-2595
Epoch: 0 Iteration: 1 : Loss: 0.7889 (3.319 sec)
Epoch: 0 Iteration: 2 : Loss: 0.9898 (2.518 sec)
Epoch: 0 Iteration: 3 : Loss: 1.1783 (2.657 sec)
Epoch: 0 Iteration: 4 : Loss: 0.7721 (2.521 sec)
Epoch: 0 Iteration: 5 : Loss: 0.6853 (2.457 sec)
Epoch: 0 Iteration: 6 : Loss: 0.8940 (2.557 sec)
Epoch: 0 Iteration: 7 : Loss: 0.6697 (2.661 sec)
Epoch: 0 Iteration: 8 : Loss: 0.9194 (2.526 sec)
Epoch: 0 Iteration: 9 : Loss: 1.2226 (2.648 sec)
Epoch: 0 Iteration: 10 : Loss: 0.8992 (2.701 sec)
Epoch: 0 Iteration: 11 : Loss: 0.7281 (2.758 sec)
Epoch: 0 Iteration: 12 : Loss: 0.9056 (2.583 sec)
Epoch: 0 Iteration: 13 : Loss: 0.6573 (2.549 sec)
Epoch: 0 Iteration: 14 : Loss: 0.9445 (2.627 sec)
Epoch: 0 Iteration: 15 : Loss: 1.1703 (2.727 sec)
Epoch: 0 Iteration: 16 : Loss: 0.8683 (2.778 sec)
Epoch: 0 Iteration: 17 : Loss: 0.7193 (2.677 sec)
Epoch: 0 Iteration: 18 : Loss: 0.9975 (2.668 sec)
Epoch:

KeyboardInterrupt: 

In [33]:
# How much data to evaluate
#number_of_batches = len(x_train_text)/batch_size
number_of_validation_batches = math.floor(len(x_test_text)/batch_size)
ckpt = tf.train.get_checkpoint_state(logs_dir)                                     

#Something is totally off here.

# Write TF Model Evaluation Code here. 
with tf.Session(graph=graph, config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True)) as session:
    
    saver = tf.train.Saver(tf.global_variables()) 
                                                                                                  
    session.run(tf.global_variables_initializer())
    
    saver.restore(session, ckpt.model_checkpoint_path)

    total_loss = 0
    for step, eval_feed_dict in enumerate(eval_input_fn()):
        #print(model_output.eval(eval_feed_dict))
    
        valid_loss, check, mae = session.run([
                loss, model_output, mean_absolute_error_salary_scale], feed_dict=eval_feed_dict)
        #print("Batch: %d Mean: %.4f" % (step, mae))
        #print(check)
        total_loss += mae
        print ("Loss over first %s observations = %s" % ((step + 1) * batch_size, total_loss / (step + 1)))

INFO:tensorflow:Restoring parameters from logs/bagofwords/2017-06-15-23-13-33-6945
Loss over first 256 observations = 9814.44433594
Loss over first 512 observations = 9377.17431641
Loss over first 768 observations = 9424.296875
Loss over first 1024 observations = 9380.40161133
Loss over first 1280 observations = 9531.45234375
Loss over first 1536 observations = 9515.10026042
Loss over first 1792 observations = 9370.55678013
Loss over first 2048 observations = 9382.71057129
Loss over first 2304 observations = 9390.62641059
Loss over first 2560 observations = 9501.98544922
Loss over first 2816 observations = 9495.34321733
Loss over first 3072 observations = 9546.7035319
Loss over first 3328 observations = 9496.75165264
Loss over first 3584 observations = 9519.98646763
Loss over first 3840 observations = 9479.69153646
Loss over first 4096 observations = 9517.35900879
Loss over first 4352 observations = 9592.41044347
Loss over first 4608 observations = 9602.99104818
Loss over first 4864 ob

In [75]:
#model_output.eval()
eval_generator = eval_input_fn()

with sess:
    saver = tf.train.Saver(tf.global_variables())                                                                                               
    sess.run(tf.global_variables_initializer())
    sess.run(model_output, feed_dict=next(eval_generator))

RuntimeError: Attempted to use a closed Session.

In [114]:
%matplotlib inline

def print_results(model):
    y_vals = [value for value in model.predict(input_fn=eval_input_fn)]
    x_vals = X_test[LABEL_COLUMN]


    plt.scatter(x_vals, y_vals)
    plt.show()

    print(x_vals-y_vals)
    #print(sorted([set(y_vals)]))

    print("Mean absolute error: %s" % 
          mean_absolute_error_salary_scale(y_vals, x_vals))
    
    
print_results(m)

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.


NotFittedError: Couldn't find trained model at /tmp/tmpc4vvis3j.